In [1313]:
import tensorflow as tf
import numpy as np
import ipycanvas
import ipywidgets
import pandas as pd

In [1314]:
tf.__version__

'2.7.0'

In [1315]:
tf.config.set_visible_devices([], 'GPU')

In [1316]:
n_particle = 512
initial_position = np.random.rand(n_particle, 2)
max_radius = 2
min_radius = 1
radius = (max_radius - min_radius) * np.random.rand(n_particle, 1) + min_radius
radius = radius.reshape(n_particle, 1)
shear_angle = 0.0
volumn = np.square(radius).sum() * np.pi
target_volumn = 0.90
radius = np.sqrt(target_volumn / volumn) * radius

In [1317]:
pos = tf.Variable(initial_value=initial_position, dtype=tf.float32)
shear_angle = tf.Variable(initial_value=shear_angle, dtype=pos.dtype)
radius = tf.constant(radius, dtype=pos.dtype)
half = tf.constant(0.5, dtype=pos.dtype)
zero = tf.constant(0.0, dtype=pos.dtype)
epsilon = tf.constant(1e-12, dtype=pos.dtype)

In [1318]:
width = 600
def visual(canvas, pos, radius, shear_angle=0.0, periodic=True):
    """Plots the particles."""
    canvas.clear()
    canvas.fill_style = "blue"
    canvas.global_alpha = 0.12
    n_particles = pos.shape[0]
    scale = width
    if periodic:
        shift = [-1, 0, 1]
    else:
        shift = [0] # no shift for open boundary conditions.
    with ipycanvas.hold_canvas(canvas):
        for i in range(n_particles):
            for x_shift in shift:
                for y_shift in shift:
                    canvas.fill_circle(scale * (pos[i, 0] + x_shift + shear_angle * (pos[i, 1] + y_shift)), scale * (pos[i, 1] + y_shift), scale * radius[i])

In [1319]:
canvas = ipycanvas.Canvas(width=width, height=width)

In [1320]:
display(canvas)

Canvas(height=600, width=600)

In [1321]:
visual(canvas, pos.numpy(), radius.numpy())

In [1322]:
def pairwise_potential(pos1, pos2, r1, r2):
    """Calculates the pairwise potential energy."""
    pos = tf.math.mod(pos1 - pos2 + half, 1.0) - half
    pos_square_sum = tf.reduce_sum(tf.math.square(pos))
    distance = tf.math.sqrt(pos_square_sum + epsilon)
    cutoff = r1 + r2
    return half * tf.math.square(tf.math.maximum(epsilon, cutoff - distance))
    
# @tf.function    
def energy(position: tf.Tensor, radius: tf.Tensor):
    """Calculates the total potential energy."""
    total_energy = tf.constant(0.0, dtype=position.dtype)
    # print("Tracing")
    for i in range(n_particle):
        for j in range(i+1, n_particle):
            total_energy += pairwise_potential(position[i], position[j], radius[i], radius[j])
    return total_energy

@tf.function    
def energy_matrix(position: tf.Tensor, radius: tf.Tensor):
    """Calculates the total potential energy."""
    x = position[:, 0:1]
    y = position[:, 1:2]
    radius = tf.reshape(radius, shape=x.shape)
    dx = tf.math.mod(x - tf.transpose(x) + half, 1.0) - half
    dy = tf.math.mod(y - tf.transpose(y) + half, 1.0) - half
    distance_square_matrix = tf.square(dx) + tf.square(dy)
    cutoff_matrix = radius + tf.transpose(radius)
    distance_matrix = tf.math.sqrt(epsilon + distance_square_matrix)
    energy_matrix = half * tf.math.square(tf.math.maximum(epsilon, cutoff_matrix - distance_matrix))
    energy = half * (tf.reduce_sum(energy_matrix) - tf.linalg.trace(energy_matrix))
    return energy

@tf.function    
def energy_shear_matrix(position: tf.Tensor, radius: tf.Tensor, shear_angle: tf.Tensor):
    """Calculates the total potential energy."""
    x = position[:, 0:1]
    y = position[:, 1:2]
    radius = tf.reshape(radius, shape=x.shape)
    dx = tf.math.mod(x - tf.transpose(x) + half, 1.0) - half
    dy = tf.math.mod(y - tf.transpose(y) + half, 1.0) - half
    dx = dx + shear_angle * dy
    distance_square_matrix = tf.square(dx) + tf.square(dy)
    cutoff_matrix = radius + tf.transpose(radius)
    distance_matrix = tf.math.sqrt(epsilon + distance_square_matrix)
    energy_matrix = half * tf.math.square(tf.math.maximum(epsilon, cutoff_matrix - distance_matrix))
    energy = half * (tf.reduce_sum(energy_matrix) - tf.linalg.trace(energy_matrix))
    return energy

In [1323]:
pairwise_potential(pos[0], pos[1], radius[0], radius[1])

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([5.e-25], dtype=float32)>

In [1324]:
output = ipywidgets.widgets.Output()

In [1325]:
display(output)
display(canvas)

Output()

Canvas(height=600, width=600)

In [1326]:
schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2, decay_steps=100, decay_rate=0.9, 
)
# optimizer_sgd = tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=0.99)
optimizer = tf.keras.optimizers.Adam(learning_rate=schedule)
# optimizer = tf.keras.optimizers.Adagrad(learning_rate=1e-1)
# optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-3, momentum=0.9)
steps = 1000
for step in range(steps):
    with tf.GradientTape() as tape:
        total_energy = energy_shear_matrix(pos, radius, shear_angle)
    if step < steps / 2:
        var_list = [pos]
    else:
        var_list = [pos, shear_angle]
    grad = tape.gradient(total_energy, var_list)
    optimizer.apply_gradients(zip(grad, var_list))
    with ipycanvas.hold_canvas(canvas):
        visual(canvas, pos.numpy(), radius.numpy(), shear_angle=shear_angle.numpy())
    with output:
        output.clear_output()
        print("Shear angle = %.2f" % shear_angle.numpy())
        

In [1327]:
def contection_matrix(position: tf.Tensor, radius: tf.Tensor, shear_angle: tf.Tensor, periodic: bool):
    """Calculates the connections between particles."""
    x = position[:, 0:1]
    y = position[:, 1:2]
    radius = tf.reshape(radius, shape=x.shape)
    if periodic:
        dx = tf.math.mod(x - tf.transpose(x) + half, 1.0) - half
        dy = tf.math.mod(y - tf.transpose(y) + half, 1.0) - half
    else:
        dx = x - tf.transpose(x)
        dy = y - tf.transpose(y)
    dy = dy + shear_angle * dx
    distance_square_matrix = tf.square(dx) + tf.square(dy)
    cutoff_matrix = radius + tf.transpose(radius)
    distance_matrix = tf.math.sqrt(epsilon + distance_square_matrix)
    connection = tf.cast(cutoff_matrix > distance_matrix, dtype=pos.dtype)
    return connection - tf.linalg.diag(tf.linalg.diag_part(connection))

In [1328]:
connection = contection_matrix(pos, radius, shear_angle, periodic=False)

In [1329]:
final_position = pos.numpy()
final_position = np.mod(final_position, 1.0)
final_position[:, 0] += shear_angle.numpy() * final_position[:, 1]
final_position = np.mod(final_position, 1.0)
connection = contection_matrix(final_position, radius, shear_angle=0.0, periodic=False)

In [1330]:
visual(canvas, final_position, radius.numpy(), periodic=False)
display(canvas)

Canvas(height=600, width=600)

In [1331]:
np.savetxt("connection.csv", connection.numpy(), fmt='%.1e')
np.savetxt("position.csv", final_position, )
np.savetxt("radius.csv", radius.numpy(), )